In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_3_Thermal_Plants/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 21:12:12.422622


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Parameters:

Let's denote the following parameters:

- \( C_{SU}^i \): Startup cost for power plant \( i \)
- \( C_{SD}^i \): Shutdown cost for power plant \( i \)
- \( C_F^i \): Fixed cost for power plant \( i \)
- \( C_V^{it} \): Variable cost for power plant \( i \) at time \( t \)
- \( P_{min}^i \): Minimum power output for power plant \( i \)
- \( P_{max}^i \): Maximum power output for power plant \( i \)
- \( P_{inc}^i \): Maximum power increment for power plant \( i \)
- \( P_{dec}^i \): Maximum power decrement for power plant \( i \)
- \( D^t \): Power demand at time \( t \)

Decision variables:

Let's denote the following decision variables:

- \( u_t^i \in \{0, 1\} \): Binary variable indicating if power plant \( i \) is on at time \( t \)
- \( P_t^i \): Power output of power plant \( i \) at time \( t \)

Objective function:

The objective is to minimize the total cost, including startup, shutdown, fixed, and variable costs:

\[ \text{min} \sum_{i=1}^{6} \sum_{t=1}^{15} \left[ C_F^i u_t^i + C_V^{it} P_t^i + C_{SU}^i (u_t^i - u_{t-1}^i)^+ + C_{SD}^i (u_{t-1}^i - u_t^i)^+ \right] \]

Constraints:

1. Power output limits:
\[ P_{min}^i u_t^i \leq P_t^i \leq P_{max}^i u_t^i \; \forall i \in \{1, 2, \ldots, 6\}, t \in \{1, 2, \ldots, 15\} \]

2. Power increment and decrement limits:
\[ P_t^i - P_{t-1}^i \leq P_{inc}^i u_{t-1}^i + M (1 - u_t^i) \; \forall i \in \{1, 2, \ldots, 6\}, t \in \{2, 3, \ldots, 15\} \]
\[ P_{t-1}^i - P_t^i \leq P_{dec}^i u_t^i + M (1 - u_{t-1}^i) \; \forall i \in \{1, 2, \ldots, 6\}, t \in \{2, 3, \ldots, 15\} \]

3. Power demand satisfaction:
\[ \sum_{i=1}^{6} P_t^i \geq D^t \; \forall t \in \{1, 2, \ldots, 15\} \]

4. Security constraint:
\[ \sum_{i=1}^{6} P_t^i \geq 1.1 D^t \; \forall t \in \{1, 2, \ldots, 15\} \]

Here, \( M \) is a large constant that can be set to the maximum possible power output of a power plant. The notation \( (x)^+ \) represents the positive part of \( x \), i.e., \( (x)^+ = \max\{x, 0\} \).

This optimization model is a mixed-integer linear program (MILP) that can be solved using off-the-shelf optimization solvers such as CPLEX, Gurobi, or GLPK.

In [6]:
print(response_1_text)

Parameters:

Let's denote the following parameters:

- \( C_{SU}^i \): Startup cost for power plant \( i \)
- \( C_{SD}^i \): Shutdown cost for power plant \( i \)
- \( C_F^i \): Fixed cost for power plant \( i \)
- \( C_V^{it} \): Variable cost for power plant \( i \) at time \( t \)
- \( P_{min}^i \): Minimum power output for power plant \( i \)
- \( P_{max}^i \): Maximum power output for power plant \( i \)
- \( P_{inc}^i \): Maximum power increment for power plant \( i \)
- \( P_{dec}^i \): Maximum power decrement for power plant \( i \)
- \( D^t \): Power demand at time \( t \)

Decision variables:

Let's denote the following decision variables:

- \( u_t^i \in \{0, 1\} \): Binary variable indicating if power plant \( i \) is on at time \( t \)
- \( P_t^i \): Power output of power plant \( i \) at time \( t \)

Objective function:

The objective is to minimize the total cost, including startup, shutdown, fixed, and variable costs:

\[ \text{min} \sum_{i=1}^{6} \sum_{t=1}^{15} \lef

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

To solve this optimization problem using Pyomo, we first need to install the necessary packages. If you haven't done so already, you can install Pyomo and a solver such as GLPK with the following commands:

```bash
pip install pyomo
pip install pyomo.solvers
pip install pyomo.solvers.plugins.glpk
```

Now, let's write the Pyomo code for the given optimization problem. For simplicity, I will use sample data for the parameters and a constant value for the variable cost.

```python
import pyomo.environ as pyo

# Parameters (sample data)
num_plants = 6
num_time_periods = 15

plant_startup_costs = [10, 15, 12, 18, 14, 16]
plant_shutdown_costs = [8, 10, 9, 12, 11, 13]
plant_fixed_costs = [4, 6, 5, 7, 6, 8]
plant_variable_cost = 2  # Constant variable cost for all plants and time periods
plant_min_power_outputs = [20, 30, 25, 35, 30, 40]
plant_max_power_outputs = [100, 150, 125, 175, 150, 200]
plant_inc_power_outputs = [30, 40, 35, 45, 40, 50]
plant_dec_power_outputs = [20, 30, 25, 35, 30, 40]
demand = [100, 120, 150, 180, 200, 220, 250, 280, 300, 320, 350, 380, 400, 420, 450]
M = max(plant_max_power_outputs)

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Binary variables (on/off status of plants)
model.u = pyo.Var(range(1, num_plants + 1), range(1, num_time_periods + 1), domain=pyo.Binary)

# Continuous variables (power outputs of plants)
model.P = pyo.Var(range(1, num_plants + 1), range(1, num_time_periods + 1), domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return sum(
        plant_fixed_costs[i - 1] * model.u[i, t] + plant_variable_cost * model.P[i, t] +
        plant_startup_costs[i - 1] * pyo.max_((model.u[i, t] - model.u[i, t - 1])) +
        plant_shutdown_costs[i - 1] * pyo.max_((model.u[i, t - 1] - model.u[i, t]))
        for i in range(1, num_plants + 1) for t in range(1, num_time_periods + 1)
    )

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def power_output_limits_rule(model, i, t):
    return plant_min_power_outputs[i - 1] * model.u[i, t] <= model.P[i, t] <= plant_max_power_outputs[i - 1] * model.u[i, t]

model.power_output_limits = pyo.Constraint(range(1, num_plants + 1), range(1, num_time_periods + 1), rule=power_output_limits_rule)

def power_increment_decrement_limits_rule(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    return model.P[i, t] - model.P[i, t - 1] <= plant_inc_power_outputs[i - 1] * model.u[i, t - 1] + M * (1 - model.u[i, t]), \
           model.P[i, t - 1] - model.P[i, t] <= plant_dec_power_outputs[i - 1] * model.u[i, t] + M * (1 - model.u[i, t - 1])

model.power_increment_decrement_limits = pyo.Constraint(range(1, num_plants + 1), range(1, num_time_periods + 1), rule=power_increment_decrement_limits_rule)

def power_demand_satisfaction_rule(model, t):
    return sum(model.P[i, t] for i in range(1, num_plants + 1)) >= demand[t - 1]

model.power_demand_satisfaction = pyo.Constraint(range(1, num_time_periods + 1), rule=power_demand_satisfaction_rule)

def security_constraint_rule(model, t):
    return sum(model.P[i, t] for i in range(1, num_plants + 1)) >= 1.1 * demand[t - 1]

model.security_constraint = pyo.Constraint(range(1, num_time_periods + 1), rule=security_constraint_rule)

# Solve the optimization problem
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Plant statuses:")
for i in range(1, num_plants + 1):
    for t in range(1, num_time_periods + 1):
        print(f"Plant {i}, time {t}: status = {pyo.value(model.u[i, t])}, power output = {pyo.value(model.P[i, t])}")
```

In [10]:
print(response_2_text)

To solve this optimization problem using Pyomo, we first need to install the necessary packages. If you haven't done so already, you can install Pyomo and a solver such as GLPK with the following commands:

```bash
pip install pyomo
pip install pyomo.solvers
pip install pyomo.solvers.plugins.glpk
```

Now, let's write the Pyomo code for the given optimization problem. For simplicity, I will use sample data for the parameters and a constant value for the variable cost.

```python
import pyomo.environ as pyo

# Parameters (sample data)
num_plants = 6
num_time_periods = 15

plant_startup_costs = [10, 15, 12, 18, 14, 16]
plant_shutdown_costs = [8, 10, 9, 12, 11, 13]
plant_fixed_costs = [4, 6, 5, 7, 6, 8]
plant_variable_cost = 2  # Constant variable cost for all plants and time periods
plant_min_power_outputs = [20, 30, 25, 35, 30, 40]
plant_max_power_outputs = [100, 150, 125, 175, 150, 200]
plant_inc_power_outputs = [30, 40, 35, 45, 40, 50]
plant_dec_power_outputs = [20, 30, 25, 35, 30, 40

### Code Executability

In [11]:
import pyomo.environ as pyo

# Parameters (sample data)
num_plants = 6
num_time_periods = 15

plant_startup_costs = [10, 15, 12, 18, 14, 16]
plant_shutdown_costs = [8, 10, 9, 12, 11, 13]
plant_fixed_costs = [4, 6, 5, 7, 6, 8]
plant_variable_cost = 2  # Constant variable cost for all plants and time periods
plant_min_power_outputs = [20, 30, 25, 35, 30, 40]
plant_max_power_outputs = [100, 150, 125, 175, 150, 200]
plant_inc_power_outputs = [30, 40, 35, 45, 40, 50]
plant_dec_power_outputs = [20, 30, 25, 35, 30, 40]
demand = [100, 120, 150, 180, 200, 220, 250, 280, 300, 320, 350, 380, 400, 420, 450]
M = max(plant_max_power_outputs)

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Binary variables (on/off status of plants)
model.u = pyo.Var(range(1, num_plants + 1), range(1, num_time_periods + 1), domain=pyo.Binary)

# Continuous variables (power outputs of plants)
model.P = pyo.Var(range(1, num_plants + 1), range(1, num_time_periods + 1), domain=pyo.NonNegativeReals)

# Objective function
def obj_rule(model):
    return sum(
        plant_fixed_costs[i - 1] * model.u[i, t] + plant_variable_cost * model.P[i, t] +
        plant_startup_costs[i - 1] * pyo.max_((model.u[i, t] - model.u[i, t - 1])) +
        plant_shutdown_costs[i - 1] * pyo.max_((model.u[i, t - 1] - model.u[i, t]))
        for i in range(1, num_plants + 1) for t in range(1, num_time_periods + 1)
    )

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def power_output_limits_rule(model, i, t):
    return plant_min_power_outputs[i - 1] * model.u[i, t] <= model.P[i, t] <= plant_max_power_outputs[i - 1] * model.u[i, t]

model.power_output_limits = pyo.Constraint(range(1, num_plants + 1), range(1, num_time_periods + 1), rule=power_output_limits_rule)

def power_increment_decrement_limits_rule(model, i, t):
    if t == 1:
        return pyo.Constraint.Skip
    return model.P[i, t] - model.P[i, t - 1] <= plant_inc_power_outputs[i - 1] * model.u[i, t - 1] + M * (1 - model.u[i, t]), \
           model.P[i, t - 1] - model.P[i, t] <= plant_dec_power_outputs[i - 1] * model.u[i, t] + M * (1 - model.u[i, t - 1])

model.power_increment_decrement_limits = pyo.Constraint(range(1, num_plants + 1), range(1, num_time_periods + 1), rule=power_increment_decrement_limits_rule)

def power_demand_satisfaction_rule(model, t):
    return sum(model.P[i, t] for i in range(1, num_plants + 1)) >= demand[t - 1]

model.power_demand_satisfaction = pyo.Constraint(range(1, num_time_periods + 1), rule=power_demand_satisfaction_rule)

def security_constraint_rule(model, t):
    return sum(model.P[i, t] for i in range(1, num_plants + 1)) >= 1.1 * demand[t - 1]

model.security_constraint = pyo.Constraint(range(1, num_time_periods + 1), rule=security_constraint_rule)

# Solve the optimization problem
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Plant statuses:")
for i in range(1, num_plants + 1):
    for t in range(1, num_time_periods + 1):
        print(f"Plant {i}, time {t}: status = {pyo.value(model.u[i, t])}, power output = {pyo.value(model.P[i, t])}")

2024-05-22 21:18:53,962 ERROR pyomo.core: Rule failed when generating expression for Objective obj with index None:
AttributeError: module 'pyomo.environ' has no attribute 'max_'
2024-05-22 21:18:53,963 ERROR pyomo.core: Constructing component 'obj' from data=None failed:
AttributeError: module 'pyomo.environ' has no attribute 'max_'


AttributeError: module 'pyomo.environ' has no attribute 'max_'

### Solution Correctness